In [1]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import json
import requests
import time

In [3]:
# Renseigner ici la première ligne et la dernière ligne de la base adresses qui vont être utilisées
row_first = 979200
row_last = 1058649

In [4]:
# Chargement du jeu de données
df = pd.read_csv('valeursfoncieres-2020.txt',sep="|",decimal=",")
df.head()
#print(df.columns)

C:\Users\Charles\anaconda3\envs\CDV-perso\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (14,18,23,24,26,28,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Surface Carrez du 5eme lot,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,07/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1061.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,BT,NaN,85.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1115.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1940.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,02/01/2020,Vente,...,NaN,0,NaN,NaN,NaN,NaN,NaN,T,NaN,1148.0


In [5]:
# focus on the address datas
ad = df[['No voie', 'Type de voie', 'Voie', 'Code postal','Commune']]
ad.shape

(2459560, 5)

In [6]:
ad.head()

,No voie,Type de voie,Voie,Code postal,Commune
0,NaN,NaN,FORTUNAT,1250.0,CEYZERIAT
1,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ
2,NaN,NaN,BOIS DU CHAMP RION,1290.0,LAIZ
3,NaN,NaN,EN COROBERT,1290.0,LAIZ
4,NaN,NaN,TERRES DES CINQ SAULES,1290.0,LAIZ


In [7]:
# On supprime les adresses dupliquées pour éviter d'interroger les API plusieurs fois avec exactement la même adresse
ad = ad.drop_duplicates()
ad.shape

(1058649, 5)

In [8]:
# Sélection des adresses à partir de row_first et row_last définies plus haut
selec = ad.iloc[row_first:row_last, :]
selec.shape

(79449, 5)

In [9]:
selec.head(10)

,No voie,Type de voie,Voie,Code postal,Commune
2225019,53.0,AV,GALLIENI,92400.0,COURBEVOIE
2225021,2.0,AV,GEORGES CLEMENCEAU,92500.0,RUEIL-MALMAISON
2225022,31.0,RUE,JEAN JAURES,92800.0,PUTEAUX
2225027,7.0,RUE,DIDEROT,92000.0,NANTERRE
2225029,4.0,ALL,DES CHARMES,92500.0,RUEIL-MALMAISON
2225038,83.0,RUE,DE VERDUN,92150.0,SURESNES
2225057,22.0,RUE,VOLTAIRE,92500.0,RUEIL-MALMAISON
2225058,66.0,RUE,SEGOFFIN,92400.0,COURBEVOIE
2225069,14.0,RUE,DE LA MELONNIERE,92500.0,RUEIL-MALMAISON
2225072,243.0,BD,SAINT DENIS,92400.0,COURBEVOIE


In [10]:
# Traitement des codes postaux (transformés en INT sauf les NA)

selec['Code postal'] = selec['Code postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune
2225019,53.0,AV,GALLIENI,92400,COURBEVOIE
2225021,2.0,AV,GEORGES CLEMENCEAU,92500,RUEIL-MALMAISON
2225022,31.0,RUE,JEAN JAURES,92800,PUTEAUX
2225027,7.0,RUE,DIDEROT,92000,NANTERRE
2225029,4.0,ALL,DES CHARMES,92500,RUEIL-MALMAISON


In [11]:
# On formate les adresses
# tools functions : nan values management
def escapeNan(row):
    val = str(row)
    if val == "nan":
        val = " "
    else:
        val = val.replace('.0','') # if from float values
        val = val + " "
    return val

In [12]:
# Création d'un champ "Ville" pour interroger les coordonnées GPS lorsque le champ Adresse ne donne rien
selec['Ville'] = selec['Code postal'] + " " + selec['Commune']
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Ville
2225019,53.0,AV,GALLIENI,92400,COURBEVOIE,92400 COURBEVOIE
2225021,2.0,AV,GEORGES CLEMENCEAU,92500,RUEIL-MALMAISON,92500 RUEIL-MALMAISON
2225022,31.0,RUE,JEAN JAURES,92800,PUTEAUX,92800 PUTEAUX
2225027,7.0,RUE,DIDEROT,92000,NANTERRE,92000 NANTERRE
2225029,4.0,ALL,DES CHARMES,92500,RUEIL-MALMAISON,92500 RUEIL-MALMAISON


In [13]:
# Création du champ "addressName" qui sera utilisée dans la requête API
selec['Adresse'] = selec.apply(lambda row: 
    escapeNan(row['No voie'])
    + escapeNan(row['Type de voie'])         
    + escapeNan(row['Voie'])
    + escapeNan(row['Code postal']).zfill(5) # pour respecter le format des codes postaux
    + str(row['Commune'])
    , axis = 1).str.strip()

In [14]:
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Ville,Adresse
2225019,53.0,AV,GALLIENI,92400,COURBEVOIE,92400 COURBEVOIE,53 AV GALLIENI 92400 COURBEVOIE
2225021,2.0,AV,GEORGES CLEMENCEAU,92500,RUEIL-MALMAISON,92500 RUEIL-MALMAISON,2 AV GEORGES CLEMENCEAU 92500 RUEIL-MALMAISON
2225022,31.0,RUE,JEAN JAURES,92800,PUTEAUX,92800 PUTEAUX,31 RUE JEAN JAURES 92800 PUTEAUX
2225027,7.0,RUE,DIDEROT,92000,NANTERRE,92000 NANTERRE,7 RUE DIDEROT 92000 NANTERRE
2225029,4.0,ALL,DES CHARMES,92500,RUEIL-MALMAISON,92500 RUEIL-MALMAISON,4 ALL DES CHARMES 92500 RUEIL-MALMAISON


In [15]:
selec.iloc[0:200, :]

,No voie,Type de voie,Voie,Code postal,Commune,Ville,Adresse
2225019,53.0,AV,GALLIENI,92400,COURBEVOIE,92400 COURBEVOIE,53 AV GALLIENI 92400 COURBEVOIE
2225021,2.0,AV,GEORGES CLEMENCEAU,92500,RUEIL-MALMAISON,92500 RUEIL-MALMAISON,2 AV GEORGES CLEMENCEAU 92500 RUEIL-MALMAISON
2225022,31.0,RUE,JEAN JAURES,92800,PUTEAUX,92800 PUTEAUX,31 RUE JEAN JAURES 92800 PUTEAUX
2225027,7.0,RUE,DIDEROT,92000,NANTERRE,92000 NANTERRE,7 RUE DIDEROT 92000 NANTERRE
2225029,4.0,ALL,DES CHARMES,92500,RUEIL-MALMAISON,92500 RUEIL-MALMAISON,4 ALL DES CHARMES 92500 RUEIL-MALMAISON
...,...,...,...,...,...,...,...
2225535,36.0,BD,DE LA REPUBLIQUE,92250,LA GARENNE-COLOMBES,92250 LA GARENNE-COLOMBES,36 BD DE LA REPUBLIQUE 92250 LA GARENNE-COLOMBES
2225537,9.0,AV,MICHEL RICARD,92270,BOIS-COLOMBES,92270 BOIS-COLOMBES,9 AV MICHEL RICARD 92270 BOIS-COLOMBES
2225538,5.0,AV,MICHEL RICARD,92270,BOIS-COLOMBES,92270 BOIS-COLOMBES,5 AV MICHEL RICARD 92270 BOIS-COLOMBES
2225539,58.0,AV,DES GRESILLONS,92600,ASNIERES-SUR-SEINE,92600 ASNIERES-SUR-SEINE,58 AV DES GRESILLONS 92600 ASNIERES-SUR-SEINE


In [16]:
selec['Ville'] = selec['Ville'].replace('-', ' ', regex=True).astype(str)
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Ville,Adresse
2225019,53.0,AV,GALLIENI,92400,COURBEVOIE,92400 COURBEVOIE,53 AV GALLIENI 92400 COURBEVOIE
2225021,2.0,AV,GEORGES CLEMENCEAU,92500,RUEIL-MALMAISON,92500 RUEIL MALMAISON,2 AV GEORGES CLEMENCEAU 92500 RUEIL-MALMAISON
2225022,31.0,RUE,JEAN JAURES,92800,PUTEAUX,92800 PUTEAUX,31 RUE JEAN JAURES 92800 PUTEAUX
2225027,7.0,RUE,DIDEROT,92000,NANTERRE,92000 NANTERRE,7 RUE DIDEROT 92000 NANTERRE
2225029,4.0,ALL,DES CHARMES,92500,RUEIL-MALMAISON,92500 RUEIL MALMAISON,4 ALL DES CHARMES 92500 RUEIL-MALMAISON


In [17]:
# Instanciation de la BDD de correspondance entre Code Postal / Communes et Code INSEE (nécessaire pour code IRIS)
# La BDD se trouve ici : https://data.opendatasoft.com/explore/dataset/code-postal-code-insee-2015@public/download/?format=csv&timezone=Europe/Berlin&lang=fr&use_labels_for_header=true&csv_separator=%3B
communes = pd.read_csv('databases/code-postal-code-insee-2015.csv', sep = ";")
communes.head()

,Geo Point,Geo Shape,CODE_COM,INSEE_COM,NOM_COM,STATUT,X_CENTROID,Y_CENTROID,Z_MOYEN,SUPERFICIE,POPULATION,CODE_CANT,CODE_ARR,CODE_DEPT,NOM_DEPT,Code_postal,Nom_commune,coordonnees_gps,Unnamed: 18
0,"49.0097923668,0.696073334129","{""type"": ""Polygon"", ""coordinates"": [[[0.682900...",49,27049,BEAUMESNIL,Commune simple,531927,6881409,162,1264,562,2.0,2.0,27,EURE,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297",NaN
1,"49.0521199676,0.412267826158","{""type"": ""Polygon"", ""coordinates"": [[[0.409543...",547,27547,SAINT-GERMAIN-LA-CAMPAGNE,Commune simple,510734,6886673,176,2231,895,3.0,2.0,27,EURE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157",NaN
2,"48.7896521776,0.276557382012","{""type"": ""Polygon"", ""coordinates"": [[[0.260047...",347,61347,RESENLIEU,Commune simple,500194,6858135,231,504,210,21.0,2.0,61,ORNE,61230.0,RESENLIEU,"48.7900887821,0.276930426076",NaN
3,"48.8470682746,0.643176111978","{""type"": ""Polygon"", ""coordinates"": [[[0.628763...",359,27359,JUIGNETTES,Commune simple,527172,6863726,228,1302,243,6.0,3.0,27,EURE,27250.0,JUIGNETTES,"48.8472706788,0.64345379282",NaN
4,"48.8662500091,0.559682765078","{""type"": ""Polygon"", ""coordinates"": [[[0.553791...",136,61136,COUVAINS,Commune simple,521103,6865598,236,1788,167,18.0,2.0,61,ORNE,NaN,NaN,NaN,NaN


In [18]:
# On ne garde que les colonnes qui nous intéressent 
# (On a les coordonnees gps dedans, ce sera intéressant pour faire un check des coordonnees qu'on récupère plus bas)

communes = communes[['INSEE_COM', 'NOM_COM', 'Code_postal', 'Nom_commune', 'coordonnees_gps']]
communes = communes.rename(columns = {'NOM_COM': "Commune"})
communes.head()

,INSEE_COM,Commune,Code_postal,Nom_commune,coordonnees_gps
0,27049,BEAUMESNIL,27330.0,MESNIL EN OUCHE,"49.0090520551,0.696415474297"
1,27547,SAINT-GERMAIN-LA-CAMPAGNE,27230.0,ST GERMAIN LA CAMPAGNE,"49.0522921876,0.412852228157"
2,61347,RESENLIEU,61230.0,RESENLIEU,"48.7900887821,0.276930426076"
3,27359,JUIGNETTES,27250.0,JUIGNETTES,"48.8472706788,0.64345379282"
4,61136,COUVAINS,NaN,NaN,NaN


In [19]:
# Instanciation d'un dataframe communes_ville qui ne conserve que le code INSEE, les coordonnés, et la ville
communes_ville = communes
communes_ville['Code_postal'] = communes_ville['Code_postal'].fillna(-1).astype(int).astype(str).replace('-1', np.nan)
communes_ville['Ville'] = communes_ville['Code_postal'] + " " + communes_ville['Commune']
communes_ville['Ville'] = communes_ville['Ville'].where(-communes_ville['Code_postal'].isna(), communes_ville['Commune'])
communes_ville = communes_ville.drop(columns = {'Commune', 'Code_postal', 'Nom_commune'})
communes_ville['Ville'] = communes_ville['Ville'].replace('-', ' ', regex=True).astype(str)
communes_ville.head()

,INSEE_COM,coordonnees_gps,Ville
0,27049,"49.0090520551,0.696415474297",27330 BEAUMESNIL
1,27547,"49.0522921876,0.412852228157",27230 SAINT GERMAIN LA CAMPAGNE
2,61347,"48.7900887821,0.276930426076",61230 RESENLIEU
3,27359,"48.8472706788,0.64345379282",27250 JUIGNETTES
4,61136,NaN,COUVAINS


In [20]:
# Ajout du code commune INSEE au dataframe test
selec = selec.merge(communes_ville, on = "Ville", how = "left")
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Ville,Adresse,INSEE_COM,coordonnees_gps
0,53.0,AV,GALLIENI,92400,COURBEVOIE,92400 COURBEVOIE,53 AV GALLIENI 92400 COURBEVOIE,92026,"48.8984504771,2.25570587289"
1,2.0,AV,GEORGES CLEMENCEAU,92500,RUEIL-MALMAISON,92500 RUEIL MALMAISON,2 AV GEORGES CLEMENCEAU 92500 RUEIL-MALMAISON,92063,"48.8691897371,2.1773406315"
2,31.0,RUE,JEAN JAURES,92800,PUTEAUX,92800 PUTEAUX,31 RUE JEAN JAURES 92800 PUTEAUX,92062,"48.8837094969,2.23834178314"
3,7.0,RUE,DIDEROT,92000,NANTERRE,92000 NANTERRE,7 RUE DIDEROT 92000 NANTERRE,92050,"48.8960701282,2.20671346353"
4,4.0,ALL,DES CHARMES,92500,RUEIL-MALMAISON,92500 RUEIL MALMAISON,4 ALL DES CHARMES 92500 RUEIL-MALMAISON,92063,"48.8691897371,2.1773406315"


In [21]:
selec['Commune_Lat'] = selec['coordonnees_gps'].str.split(",", expand = True)[0]
selec['Commune_Lon'] = selec['coordonnees_gps'].str.split(",", expand = True)[1]
selec = selec.drop(columns = {'coordonnees_gps'})
selec.head(15)

,No voie,Type de voie,Voie,Code postal,Commune,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon
0,53.0,AV,GALLIENI,92400,COURBEVOIE,92400 COURBEVOIE,53 AV GALLIENI 92400 COURBEVOIE,92026,48.8984504771,2.25570587289
1,2.0,AV,GEORGES CLEMENCEAU,92500,RUEIL-MALMAISON,92500 RUEIL MALMAISON,2 AV GEORGES CLEMENCEAU 92500 RUEIL-MALMAISON,92063,48.8691897371,2.1773406315
2,31.0,RUE,JEAN JAURES,92800,PUTEAUX,92800 PUTEAUX,31 RUE JEAN JAURES 92800 PUTEAUX,92062,48.8837094969,2.23834178314
3,7.0,RUE,DIDEROT,92000,NANTERRE,92000 NANTERRE,7 RUE DIDEROT 92000 NANTERRE,92050,48.8960701282,2.20671346353
4,4.0,ALL,DES CHARMES,92500,RUEIL-MALMAISON,92500 RUEIL MALMAISON,4 ALL DES CHARMES 92500 RUEIL-MALMAISON,92063,48.8691897371,2.1773406315
5,83.0,RUE,DE VERDUN,92150,SURESNES,92150 SURESNES,83 RUE DE VERDUN 92150 SURESNES,92073,48.8698479132,2.21965517625
6,22.0,RUE,VOLTAIRE,92500,RUEIL-MALMAISON,92500 RUEIL MALMAISON,22 RUE VOLTAIRE 92500 RUEIL-MALMAISON,92063,48.8691897371,2.1773406315
7,66.0,RUE,SEGOFFIN,92400,COURBEVOIE,92400 COURBEVOIE,66 RUE SEGOFFIN 92400 COURBEVOIE,92026,48.8984504771,2.25570587289
8,14.0,RUE,DE LA MELONNIERE,92500,RUEIL-MALMAISON,92500 RUEIL MALMAISON,14 RUE DE LA MELONNIERE 92500 RUEIL-MALMAISON,92063,48.8691897371,2.1773406315
9,243.0,BD,SAINT DENIS,92400,COURBEVOIE,92400 COURBEVOIE,243 BD SAINT DENIS 92400 COURBEVOIE,92026,48.8984504771,2.25570587289


In [22]:
selec['code_iris'] = ""
selec['lon'] = ""
selec['lat'] = ""
selec.head()

,No voie,Type de voie,Voie,Code postal,Commune,Ville,Adresse,INSEE_COM,Commune_Lat,Commune_Lon,code_iris,lon,lat
0,53.0,AV,GALLIENI,92400,COURBEVOIE,92400 COURBEVOIE,53 AV GALLIENI 92400 COURBEVOIE,92026,48.8984504771,2.25570587289,,,
1,2.0,AV,GEORGES CLEMENCEAU,92500,RUEIL-MALMAISON,92500 RUEIL MALMAISON,2 AV GEORGES CLEMENCEAU 92500 RUEIL-MALMAISON,92063,48.8691897371,2.1773406315,,,
2,31.0,RUE,JEAN JAURES,92800,PUTEAUX,92800 PUTEAUX,31 RUE JEAN JAURES 92800 PUTEAUX,92062,48.8837094969,2.23834178314,,,
3,7.0,RUE,DIDEROT,92000,NANTERRE,92000 NANTERRE,7 RUE DIDEROT 92000 NANTERRE,92050,48.8960701282,2.20671346353,,,
4,4.0,ALL,DES CHARMES,92500,RUEIL-MALMAISON,92500 RUEIL MALMAISON,4 ALL DES CHARMES 92500 RUEIL-MALMAISON,92063,48.8691897371,2.1773406315,,,


In [23]:
selec.shape

(79449, 13)

In [ ]:
# Get GPS Coordinates + IRIS Code from addresses

geolocator = Nominatim(user_agent="ImmoPy")

start = time.time()
etape = time.time()
print("Heure début : ")
print(time.strftime('%H:%M:%S', time.gmtime(start)))
print("\n")
n=0

for row in range(0, row_last - row_first):
    
    iris = ""
    longitude = ""
    latitude = ""
    
    adresse = selec['Adresse'][row]
    ville = selec['Ville'][row]
    code_INSEE = selec['INSEE_COM'][row]
    
    
    location = geolocator.geocode(adresse, country_codes = 'fr', timeout = 100) # On récup GPS from adresse
    
    if location is None: 
        location = geolocator.geocode(ville, country_codes = 'fr', timeout = 100) # Sinon on récup GPS from ville
        
    if location is None:                      # Sinon on prend les coordonnées GPS de la base INSEE Communes
        latitude = ""
        longitude = ""
    
    else:
        latitude = location.latitude
        longitude = location.longitude
        
        if isinstance(code_INSEE, float):     # Si pas de code commune
            print(adresse + ': pas de code commune INSEE')
            iris = ""
        elif latitude == "":               # Si pas de coordonnées GPS
            print(adresse + ": pas de coordonnees GPS")
            iris = ""
        else:                              # Sinon on interroge l'API du gouv pour récupérer le code IRIS
            result = requests.get('https://geo.api.gouv.fr/iris?lon=' + str(longitude) + '&lat=' 
                                  + str(latitude) + '&codeCommune=' + str(code_INSEE), timeout = 5)
            if result.ok == True:
                obj = json.loads(result.text)
                iris = obj['codeIris']
                #print(obj['codeIris'])
            else:
                result.close()
            print(adresse + " ===> " + str(latitude) + "," + str(longitude) + " (code IRIS = " + str(iris) + ")")
        
    selec['code_iris'][row] = iris
    selec['lon'][row] = longitude
    selec['lat'][row] = latitude
    
    n = n + 1
    
    if (n % 100 == 0):
        print("\n")
        print("Etape : " + str(n))
        print(time.strftime('%H:%M:%S', time.gmtime(time.time())))
        print("Temps pour 100 : " + str(time.time() - etape) + " sec")
        print("\n")
        etape = time.time()
        
    if (n % 10000 == 0):
        selec.iloc[(n - 10000) : n, :].to_csv(
            'export_gps_iris_' + str(row_first + n - 10000) + "-" + str(row_first + n)+'.csv')

        
end = time.time()
print("\n")
print("Heure fin : ")
print(time.strftime('%H:%M:%S', time.gmtime(end)))
print("\n")

print("Temps de calcul : ")
print(str(end - start) + ' sec')

Heure début : 
06:20:27


53 AV GALLIENI 92400 COURBEVOIE ===> 48.9054446,2.2716728 (code IRIS = 920260303)
2 AV GEORGES CLEMENCEAU 92500 RUEIL-MALMAISON ===> 48.8791246,2.1796436 (code IRIS = 920630803)
31 RUE JEAN JAURES 92800 PUTEAUX ===> 48.8840483,2.2456132 (code IRIS = 920620403)
7 RUE DIDEROT 92000 NANTERRE ===> 48.8873277,2.2104218 (code IRIS = 920500509)
4 ALL DES CHARMES 92500 RUEIL-MALMAISON ===> 48.8669815,2.1971079 (code IRIS = 920630601)
83 RUE DE VERDUN 92150 SURESNES ===> 48.8738683,2.2300827 (code IRIS = 920730102)
22 RUE VOLTAIRE 92500 RUEIL-MALMAISON ===> 48.8765241,2.1981622 (code IRIS = 920630706)
66 RUE SEGOFFIN 92400 COURBEVOIE ===> 48.8958014,2.2462267 (code IRIS = 920260502)
14 RUE DE LA MELONNIERE 92500 RUEIL-MALMAISON ===> 48.8574016,2.1889702 (code IRIS = 920630503)
243 BD SAINT DENIS 92400 COURBEVOIE ===> 48.8966609,2.2599144 (code IRIS = 920260704)
2 AV MALVESIN 92400 COURBEVOIE ===> 48.9031001,2.2704882 (code IRIS = 920260304)
9 RUE EDOUARD VAILLANT 92800

1 RUE DE BRIENNE 92500 RUEIL-MALMAISON ===> 48.8596641,2.1865937 (code IRIS = 920630502)
54 AV PUVIS DE CHAVANNES 92400 COURBEVOIE ===> 48.8993146,2.2371535 (code IRIS = 920260406)
16 RUE DU GEN DE MIRIBEL 92500 RUEIL-MALMAISON ===> 48.87778,2.1802832 (code IRIS = 920630803)
3 RUE DE STRASBOURG 92600 ASNIERES-SUR-SEINE ===> 48.906622,2.2684623 (code IRIS = 920040105)
76 RUE SOLFERINO 92700 COLOMBES ===> 48.9300883,2.2609242 (code IRIS = 920250204)


Etape : 100
06:21:20
Temps pour 100 : 52.8574013710022 sec


23 RUE PAUL DEROULEDE 92270 BOIS-COLOMBES ===> 48.9130693,2.2703142 (code IRIS = 920090108)
21 RUE PAUL DEROULEDE 92270 BOIS-COLOMBES ===> 48.9130693,2.2703142 (code IRIS = 920090108)
50 RUE PIERRE JOIGNEAUX 92270 BOIS-COLOMBES ===> 48.9105852,2.2681378 (code IRIS = 920090110)
9 RUE PARMENTIER 92600 ASNIERES-SUR-SEINE ===> 48.9092405,2.2757751 (code IRIS = 920040102)
7 ALL DE L ILE MARANTE 92700 COLOMBES ===> 48.922788,2.2543577 (code IRIS = 920250106)
9 ALL DE L ILE MARANTE 92700

29 RUE DE LA GAIETE 92700 COLOMBES ===> 48.9259571,2.2656066 (code IRIS = 920250103)
221 AV D ARGENTEUIL 92270 BOIS-COLOMBES ===> 48.9148269,2.2674892 (code IRIS = 920090109)
10 RUE GEORGES GUYNEMER 92600 ASNIERES-SUR-SEINE ===> 48.915502,2.2906834 (code IRIS = 920040302)
15 RUE ROBERT DUPONT 92600 ASNIERES-SUR-SEINE ===> 48.9216754,2.2843378 (code IRIS = 920040604)
1 RUE CLAUDE MIVIERE 92270 BOIS-COLOMBES ===> 48.9219072,2.2724462 (code IRIS = 920090102)
23 RUE DE L AIGLE 92250 LA GARENNE-COLOMBES ===> 48.9069349,2.2465748 (code IRIS = 920350108)
RUE DE L AIGLE 92250 LA GARENNE-COLOMBES ===> 48.9069349,2.2465748 (code IRIS = 920350108)
7 RUE DES BAS 92600 ASNIERES-SUR-SEINE ===> 48.9154742,2.295023 (code IRIS = 920040501)
87 RUE ROBERT SCHUMANN 92700 COLOMBES ===> 48.922788,2.2543577 (code IRIS = 920250106)
9 RUE LEONARD DE VINCI 92230 GENNEVILLIERS ===> 48.9338118,2.2890827 (code IRIS = 920360101)
36 BD DE LA REPUBLIQUE 92250 LA GARENNE-COLOMBES ===> 48.9068185,2.2449687 (code IRIS =

124 RUE PIERRE JOIGNEAUX 92270 BOIS-COLOMBES ===> 48.9105852,2.2681378 (code IRIS = 920090110)
12 RUE CHARLES DUPORT 92270 BOIS-COLOMBES ===> 48.912699,2.2597736 (code IRIS = 920090111)
18 AV CHEVREUL 92600 ASNIERES-SUR-SEINE ===> 48.9062418,2.2665255 (code IRIS = 920040104)
10 RUE ANDRE CAYRON 92600 ASNIERES-SUR-SEINE ===> 48.91587,2.2795667 (code IRIS = 920040202)
156 BD GALLIENI 92390 VILLENEUVE-LA-GARENNE ===> 48.9355177,2.3329082 (code IRIS = 920780109)
154 BD GALLIENI 92390 VILLENEUVE-LA-GARENNE ===> 48.9355177,2.3329082 (code IRIS = 920780109)
72 RUE SAINT DENIS 92700 COLOMBES ===> 48.924087,2.2519733 (code IRIS = 920250113)
25 RUE BASLY 92230 GENNEVILLIERS ===> 48.919318,2.2941949 (code IRIS = 920360302)
6 RUE JOSEPH LECLAINCHE 92230 GENNEVILLIERS ===> 48.931090499999996,2.2938792133185455 (code IRIS = 920360204)
20 AV DE VERDUN 1916 92250 LA GARENNE-COLOMBES ===> 48.9053796,2.2348182 (code IRIS = 920350101)
6 RUE DE SOTTEVILLE 92250 LA GARENNE-COLOMBES ===> 48.905818,2.2338187

28 ALL DU MOULIN JOLY 92700 COLOMBES ===> 48.9124813,2.2462727 (code IRIS = 920250110)
2 RUE MARIA MONTESSORI 92600 ASNIERES-SUR-SEINE ===> 48.9190306,2.3156934 (code IRIS = 920040502)
17 RUE OLYMPE DE GOUGES 92600 ASNIERES-SUR-SEINE ===> 48.9186377,2.317746 (code IRIS = 920040502)
10 BD D ACHERES 92700 COLOMBES ===> 48.922788,2.2543577 (code IRIS = 920250106)
59 AV D ARGENTEUIL 92600 ASNIERES-SUR-SEINE ===> 48.9105948,2.2890454 (code IRIS = 920040305)
44 RUE MAURICE BOKANOWSKI 92600 ASNIERES-SUR-SEINE ===> 48.9072494,2.2860856 (code IRIS = 920040402)
RUE DES COTES D AUTY 92700 COLOMBES ===> 48.922788,2.2543577 (code IRIS = 920250106)
78 RUE DES COTES D AUTY 92700 COLOMBES ===> 48.922788,2.2543577 (code IRIS = 920250106)
6 RUE ANDRE CAYRON 92600 ASNIERES-SUR-SEINE ===> 48.91587,2.2795667 (code IRIS = 920040202)
11 RUE FOSSE AUX ASTRES 92390 VILLENEUVE-LA-GARENNE ===> 48.9378124,2.3326555 (code IRIS = 920780109)
20 AV JEAN MOULIN 92390 VILLENEUVE-LA-GARENNE ===> 48.9374488,2.3319582 (co

...

13  VENELLE AUX BOIS 95770 BUHY ===> 49.1988524,1.7107661 (code IRIS = 951190000)
150 RUE DE PONTOISE 95290 L'ISLE-ADAM ===> 49.1021915,2.2171655 (code IRIS = 953130105)
4 RUE ANDRE COMMELIN 95590 NERVILLE-LA-FORET ===> 49.0886892,2.2768277 (code IRIS = 954450000)
28 ALL DES GREENS 95800 COURDIMANCHE ===> 49.0417426,2.0093016 (code IRIS = 951830000)
2 RUE DE L AULNAIE 95450 SERAINCOURT ===> 49.0355508,1.8657252 (code IRIS = 955920000)
38 RTE DES CRETES 95510 VETHEUIL ===> 49.0771841,1.6983602 (code IRIS = 956510000)
17 RUE DU BEL AIR 95340 PERSAN ===> 49.1498669,2.2885446 (code IRIS = 954870105)
77 RUE DOSCOT 95340 RONQUEROLLES ===> 49.1729007,2.2173836 (code IRIS = 955290000)
22 RUE ST DENIS 95770 MONTREUIL-SUR-EPTE ===> 49.176277999999996,1.6769457470588236 (code IRIS = 954290000)
SOUS LE MOULIN 95770 MONTREUIL-SUR-EPTE ===> 49.1768421,1.6765982 (code IRIS = 954290000)
4 RUE DES PINSONS 95610 ERAGNY-SUR-OISE: pas de code commune INSEE
33 RUE DE GISORS 95300 PONTOISE ===> 49.0526801,2

35 RUE CLEMENT ADER 95820 BRUYERES-SUR-OISE ===> 49.1585761,2.3402379 (code IRIS = 951160000)
AU DESSUS DE L AUMONE 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
LE PARADIS DES ANES 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
LE BEL ORIENT 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
LA POINTE DU BEL ORIENT 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
SUR LA GRANDE ROUTE 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
LES RIVIERES 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
LES SUREAUX 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
LE MOULIN A VENT 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
LE HAUT DES COTES 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
LE MILIEU DES COTES 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
LE BAS DES COTES 95510 VETHEUIL ===> 49.06325,1.70278 (code IRIS = 956510000)
3 RUE DU CHATEAU 95510 VILLERS-EN-ARTH

6 RUE DU RAPPORTEUR 95310 ST OUEN L AUMONE: pas de code commune INSEE
3 RUE GEORGES DUHAMEL 95300 HEROUVILLE ===> 49.1006244,2.1338498 (code IRIS = 953080000)
9001 RUE DES CHENES MAUVES 95000 CERGY ===> 49.0393135,2.0710731 (code IRIS = 951270110)
206 RUE DES CHENES BRUNS 95000 CERGY ===> 49.0527528,2.0388736 (code IRIS = 951270117)
40 RUE DE ROUEN 95300 PONTOISE ===> 49.0495072,2.0883986 (code IRIS = 955000601)
55 BD DE L'EVASION 95800 CERGY ===> 49.0488343,2.0144036 (code IRIS = 951270130)
18 VLA DES AUBADES 95800 CERGY ===> 49.0527528,2.0388736 (code IRIS = 951270117)
26 RUE PASTEUR 95310 ST OUEN L AUMONE: pas de code commune INSEE
23 RUE DE CHARS 95640 BRIGNANCOURT ===> 49.1382392,1.9426856 (code IRIS = 951100000)
7 RUE RAYMOND BERRIVIN 95800 COURDIMANCHE ===> 49.0354894,1.999555 (code IRIS = 951830000)
18 PL DE LA HALLE 95420 MAGNY-EN-VEXIN ===> 49.1554195,1.7874191 (code IRIS = 953550000)
PL DE LA HALLE 95420 MAGNY-EN-VEXIN ===> 49.1554195,1.7874191 (code IRIS = 953550000)
LES CL

18 RUE FERNAND MAIGNEL 95420 MAGNY-EN-VEXIN ===> 49.1617651,1.8035412 (code IRIS = 953550000)
LES JARDINS SUD 95610 ERAGNY-SUR-OISE: pas de code commune INSEE
8 RUE DE CONFLANS 95610 ERAGNY-SUR-OISE: pas de code commune INSEE
9002 RUE DE LA CAPITAINERIE 95290 L'ISLE-ADAM ===> 49.1133593,2.2135286 (code IRIS = 953130104)
1 RUE DE LA CAPITAINERIE 95290 L'ISLE-ADAM ===> 49.1133593,2.2135286 (code IRIS = 953130104)
2 RUE DES PENDANTS 95280 JOUY-LE-MOUTIER ===> 49.0191283,2.0370492 (code IRIS = 953230105)
14 RUE GUICHARD 95620 PARMAIN ===> 49.1142657,2.2080841 (code IRIS = 954800101)
22 AV VICTOR HUGO 95630 MERIEL ===> 49.0727867,2.1981491 (code IRIS = 953920000)
16 RUE CARNOT 95690 NESLES-LA-VALLEE ===> 49.1236067,2.1820722 (code IRIS = 954460000)
1 RUE SAINT LAZARE 95290 L'ISLE-ADAM ===> 49.1113717,2.2172175 (code IRIS = 953130101)
4 RUE DU ROSSIGNOL 95290 L'ISLE-ADAM ===> 49.1057952,2.2194515 (code IRIS = 953130101)
6 RUE DU ROSSIGNOL 95290 L'ISLE-ADAM ===> 49.1057952,2.2194515 (code IRI

42 RUE NATIONALE 95490 VAUREAL ===> 49.0293718,2.0334581 (code IRIS = 956370106)
LES TROIS ARPENTS 95540 MERY-SUR-OISE ===> 49.0634561,2.1836324 (code IRIS = 953940106)
7 RUE LAMARTINE 95540 MERY-SUR-OISE ===> 49.0583611,2.1587647 (code IRIS = 953940110)
2 RUE JEAN JAURES 95310 ST OUEN L AUMONE: pas de code commune INSEE
9002 RES DU BOIS BOULOGNE 95290 L'ISLE-ADAM ===> 49.1125294,2.2186324 (code IRIS = 953130102)
2 RUE DE LA SOURCE PERDUE 95290 L'ISLE-ADAM ===> 49.117581,2.2303097 (code IRIS = 953130102)
72 AV GASTON VERMEIRE 95340 PERSAN ===> 49.1529638,2.2723392 (code IRIS = 954870104)
6 AV DE PARIS 95590 NOINTEL ===> 49.126168,2.2872877 (code IRIS = 954520000)
7 ALL MESSAGER 95300 ENNERY ===> 49.0720192,2.1017807 (code IRIS = 952110000)
8 RUE DU BOIS THIBAULT 95760 VALMONDOIS ===> 49.0967512,2.1889394 (code IRIS = 956280000)
BOIS THIBAUT 95760 VALMONDOIS ===> 49.0925741,2.192675 (code IRIS = 956280000)
3 BD DE LA REPUBLIQUE 95420 MAGNY-EN-VEXIN ===> 49.1562072,1.7913515 (code IRIS =

In [ ]:
selec.head()

In [ ]:
selec.shape

In [ ]:
selec.to_csv('export_gps_iris_' + str(row_first) + "-" + str(row_last)+'.csv')